Redo my coursework (Future and Option) using Python - implement Black Scholes and Binomial pricing using given dataset. 
 


Assume that on April 13 the ABC stock price was $20.69.  Assume that the option on ABC stock expires in 1 month and the risk-free rate is 2.7% p.a. 

a) Compute European call and put price with strike X=19 using Black Scholes closed form solution

b)	Approximate prices of European put option with X=19 and European call option with X=19 using 5-step binomial trees. 

e)	Approximate prices of American put option with X=19 and American call option with X=19 using 5-step binomial trees. 

Note: datafile: HW5_data.xls was given in class and imported below:



In [104]:
import numpy as np
import pandas as pd
!pip install --upgrade xlrd
df = pd.read_excel('HW5_data.xls')



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Part a: 

Step 1: Compute daily return, and annual volatility (252 trading days)

Step 2: Compute price with using Black Scholes closed form solution



In [121]:
from numpy import log as ln
from math import exp, sqrt
from scipy.stats import norm
for i in range(1, len(df)): 
  df.loc[i,"Daily return"] = ln(df.loc[i,'Close']/df.loc[i-1,'Close'])
print (df.head())
sigma = df['Daily return'].std()*sqrt(252)
print('Estimate volatility annually:',round(sigma*100,3),'%')
#Assumption
S  = 20.69 
K  = 19
rf = 0.027 
T  = 1/12
d1 = (ln(S/K) + T * (rf + sigma**2/2))/(sigma*sqrt(T)) 
d2 = d1 - sigma*sqrt(T) 
C  = S * norm.cdf(d1) - K * exp(-rf*T) * norm.cdf(d2)
P  = K * exp(-rf*T) * norm.cdf(-d2) - S * norm.cdf(-d1)
print("European call option price:",round(C,3), 'and put option price:', round(P,3))

   Day  Close  Daily return
0    1  20.69           NaN
1    2  21.24      0.026236
2    3  22.08      0.038786
3    4  21.42     -0.030347
4    5  21.08     -0.016000
Estimate volatility annually: 35.486 %
European call option price: 1.954 and put option price: 0.221


Part b and c:

Step 1: Create function to compute option value 

Step 2: Assumption input 

 

In [122]:
import numpy as np
from math import exp, sqrt
import timeit
def OptionsVal(n, S, K, rf, sigma, T, PC, EA):
    dt  = T/n                    
    u   = exp(sigma*sqrt(dt)) 
    d   = 1/u        
    a   = exp(rf*dt)         
    p   = (a-d)/(u-d)   
    Ps  = np.zeros((n+1, n+1))   
    Cm  = np.zeros((n+1, n+1))
    tmp = np.zeros((2,n+1))
    for j in range(n+1):
        tmp[0,j] = S*pow(d,j)
        tmp[1,j] = S*pow(u,j)
    tot = np.unique(tmp)
    c = n
    for i in range(c+1):
        for j in range(c+1):
            Ps[i,j-c-1] = tot[(n-i)+j]
        c=c-1
    for j in range(n+1, 0, -1):
        for i in range(j):
          if (EA == 1):
            if (PC == 1):                               
                if(j == n+1):
                    Cm[i,j-1] = max(K-Ps[i,j-1], 0)     
                else:
                    Cm[i,j-1] = exp(-rf*dt) * (p*Cm[i,j] + (1-p)*Cm[i+1,j]) 
            if (PC == 0):                               
                if (j == n + 1):
                    Cm[i,j-1] = max(Ps[i,j-1]-K, 0)     
                else:
                    Cm[i,j-1] = exp(-rf*dt) * (p*Cm[i,j] + (1-p)*Cm[i+1,j]) 
          else:
            if (PC == 1):                               
                if(j == n+1):
                    Cm[i,j-1] = max(K-Ps[i,j-1], 0)     
                else:
                    Cm[i,j-1] = max(exp(-rf*dt) * (p*Cm[i,j] + (1-p)*Cm[i+1,j]), K-Ps[i,j-1]) 
            if (PC == 0):                               
                if (j == n + 1):
                    Cm[i,j-1] = max(Ps[i,j-1]-K, 0)     
                else:
                    Cm[i,j-1] = max(exp(-rf*dt) * (p*Cm[i,j] + (1-p)*Cm[i+1,j]), Ps[i,j-1]-K)
    return [Ps,Cm] 


In [123]:
S = 20.69
k = 19
rf = 2.7/100
T = 1/12
n = 5

EA = 1
PC = 0
Ps,EUCmC = OptionsVal(n,S,k,rf,sigma,T,PC,EA)
PC = 1
_,EUCmP = OptionsVal(n,S,k,rf,sigma,T,PC,EA)
print('Stock:\n',np.matrix(Ps.astype(float)),'\n')
print('European Call Option:\n',np.matrix(EUCmC.astype(float)),'\n')
print('European Put Option:\n',np.matrix(EUCmP.astype(float)),'\n')

EA = 0
PC = 0
Ps,aCmC = OptionsVal(n,S,k,rf,sigma,T,PC,EA)
PC = 1
_,aCmP = OptionsVal(n,S,k,rf,sigma,T,PC,EA)
print('American Call Option:\n',np.matrix(aCmC.astype(float)),'\n')
print('American Put Option:\n',np.matrix(aCmP.astype(float)),'\n')


Stock:
 [[20.69       21.65989062 22.67524707 23.73820064 24.85098258 26.01592868]
 [ 0.         19.76353932 20.69       21.65989062 22.67524707 23.73820064]
 [ 0.          0.         18.87856387 19.76353932 20.69       21.65989062]
 [ 0.          0.          0.         18.03321601 18.87856387 19.76353932]
 [ 0.          0.          0.          0.         17.2257213  18.03321601]
 [ 0.          0.          0.          0.          0.         16.45438474]] 

European Call Option:
 [[1.974088   2.75759341 3.70087976 4.75529295 5.85953065 7.01592868]
 [0.         1.21256897 1.84111564 2.67698293 3.68379514 4.73820064]
 [0.         0.         0.60133055 1.02846871 1.69854808 2.65989062]
 [0.         0.         0.         0.18575659 0.37660651 0.76353932]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]] 

European Put Option:
 [[0.24138605 0.06353355 0.         0.         0.         0.        ]
 [0.    